# Linear Regression

In [3]:
import findspark

In [4]:
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7/')

In [5]:
import pyspark

In [7]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder.appName('lregression').getOrCreate()

In [16]:
from pyspark.ml.regression import LinearRegression

In [17]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [18]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [39]:
lr = LinearRegression(featuresCol='features',labelCol='label',
                     predictionCol='prediction',maxIter=100)

In [21]:
## Lets fit the model
lr_model = lr.fit(training)

In [22]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [23]:
lr_model.intercept

0.14228558260358093

In [24]:
training_summary = lr_model.summary

In [25]:
## r2 tells us about variance 
training_summary.r2

0.027839179518600154

In [26]:
##Lets split the data into train and test set
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [27]:
split_object = all_data.randomSplit([0.7,0.3])

In [28]:
split_object
## it contains two list of data

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [29]:
#Best way
train_data,test_data= all_data.randomSplit([0.7,0.3])

In [31]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                147|
|   mean|  0.909091031472421|
| stddev|  9.999094310834133|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [32]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 354|
|   mean|-0.01394090877614...|
| stddev|   10.44932672191659|
|    min| -28.046018037776633|
|    max|   27.78383192005107|
+-------+--------------------+



In [40]:
correct_model = lr.fit(train_data)

In [41]:
test_results = correct_model.evaluate(test_data)

In [50]:
dt = test_results.residuals

In [63]:
dt.show()

+-------------------+
|          residuals|
+-------------------+
|-27.108828229742734|
|-19.121824945887845|
|-19.233055264733924|
|-19.069634577804567|
|-16.409534541955573|
| -20.65837090878184|
|-17.172976193647777|
|-18.906467783466866|
|-12.295549127847194|
|-14.015051269639175|
|-14.236917071630423|
|-17.508034860846276|
|-16.489663677293223|
| -8.689350168768181|
|-12.983884806366854|
|-12.050144106412203|
|-10.885100855001415|
|-14.231752712646573|
|-10.350643276764364|
| -8.290230588941094|
+-------------------+
only showing top 20 rows



In [62]:
dt.groupBy().sum().collect()

[Row(sum(residuals)=113.09615323748022)]

In [43]:
test_results.rootMeanSquaredError

10.249638123788438

In [44]:
## Now lets deploy
## lets create unlabeled data
unlabeled_data = test_data.select('features')

In [45]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [46]:
predictions = correct_model.transform(unlabeled_data)

In [47]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| -1.462650640000692|
|(10,[0,1,2,3,4,5,...|-0.5454936694838747|
|(10,[0,1,2,3,4,5,...| 0.9578416987292929|
|(10,[0,1,2,3,4,5,...| 2.8082043023518364|
|(10,[0,1,2,3,4,5,...| 0.3238755009340822|
|(10,[0,1,2,3,4,5,...|  4.877685876158539|
|(10,[0,1,2,3,4,5,...| 1.4494605805992091|
|(10,[0,1,2,3,4,5,...| 3.5469229036341874|
|(10,[0,1,2,3,4,5,...| -3.015431461569096|
|(10,[0,1,2,3,4,5,...|0.03892033848647229|
|(10,[0,1,2,3,4,5,...|0.36982917647165475|
|(10,[0,1,2,3,4,5,...| 3.7355932991434058|
|(10,[0,1,2,3,4,5,...| 3.5674405739228012|
|(10,[0,1,2,3,4,5,...| -4.083876830483016|
|(10,[0,1,2,3,4,5,...| 0.8895335278315961|
|(10,[0,1,2,3,4,5,...|0.14515720373708912|
|(10,[0,1,2,3,4,5,...|-0.7306744100142116|
|(10,[0,1,2,3,4,5,...|  2.799950347106114|
|(10,[0,1,2,3,4,5,...|-0.6887045314894634|
|(10,[0,1,2,3,4,5,...| -2.287467715302127|
+----------